In [1]:
# Install dependencies (pandas, numpy, plotly, nbformat, ipython)
import sys, subprocess
pkgs = ['pandas>=2.0.0', 'numpy', 'plotly', 'nbformat>=5.2.0', 'ipython']
subprocess.check_call([sys.executable, '-m', 'pip', 'install', *pkgs])
import pandas as pd, numpy as np, plotly
print('pandas', pd.__version__, '| numpy', np.__version__, '| plotly', plotly.__version__)

pandas 2.3.2 | numpy 2.3.2 | plotly 6.3.0


# Price + Indicators (SMA/EMA) and RSI

This notebook loads CSVs generated by `scripts/mcp_yfinance_dump.py` from the `data/` folder and plots:
- Close price with SMA/EMA overlays
- RSI(14) on a second panel

Notes:
- If the history CSV doesn't include a date column, we use the row index as X-axis.
- If `*_features.csv` is missing, we compute the same features inline.
- If dividends CSV has a date column that aligns with history, we'll plot dividend markers.

In [2]:
# Imports and plotly renderer fallback
import sys, subprocess, os
try:
    import plotly
except Exception:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'plotly'])

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Choose the safest renderer for VS Code/Jupyter
def _choose_plotly_renderer():
    # 1) Prefer VS Code renderer when running inside VS Code
    try:
        if ('VSCODE_PID' in os.environ or 'VSCODE_CWD' in os.environ) and 'vscode' in pio.renderers:
            pio.renderers.default = 'vscode'
            return
    except Exception:
        pass
    # 2) If nbformat is available, use inline notebook rendering
    try:
        import nbformat as _nb  # noqa: F401
        pio.renderers.default = 'notebook_connected'
        return
    except Exception:
        pass
    # 3) Otherwise, fall back to an iframe or browser
    try:
        if 'iframe_connected' in pio.renderers:
            pio.renderers.default = 'iframe_connected'
            return
    except Exception:
        pass
    pio.renderers.default = 'browser'

_choose_plotly_renderer()
print(f"Plotly renderer: {pio.renderers.default}")

Plotly renderer: vscode


In [3]:
# Parameters
DATA_DIR = '/Users/arturoquiroga/GITHUB/TRADE-AI/data'
SYMBOL = 'TSLA'  # change to 'MSFT' to view Microsoft, or 'AAPL' for Apple, or 'TSLA' for Tesla

In [4]:
# Helpers: load CSVs, ensure a usable X axis, compute indicators if needed
from pandas.errors import EmptyDataError
def load_history(symbol: str, base_dir: str = DATA_DIR) -> pd.DataFrame:
    path = os.path.join(base_dir, f'{symbol}_history.csv')
    if not os.path.exists(path):
        raise FileNotFoundError(f'Missing history CSV: {path}')
    try:
        if os.path.getsize(path) == 0:
            print(f'Warning: history CSV is empty: {path}')
            return pd.DataFrame()
    except OSError:
        pass
    try:
        df = pd.read_csv(path)
    except EmptyDataError:
        print(f'Warning: history CSV contains no parseable data: {path}')
        return pd.DataFrame()
    # Try to find a date-like column
    date_col = None
    for c in df.columns:
        cl = str(c).lower()
        if 'date' in cl or 'time' in cl:
            date_col = c
            break
    if date_col:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce', utc=True)
        df = df.dropna(subset=[date_col])
        df = df.sort_values(by=date_col).reset_index(drop=True)
        df = df.set_index(date_col)
    else:
        # Use integer index when no date is present
        df.index.name = 'index'
    return df

def compute_indicators(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    # Locate a close price column
    close_col = None
    for c in out.columns:
        cl = str(c).lower()
        if cl in ('close', 'adj close', 'adj_close', 'adjclose'):
            close_col = c
            break
    if close_col is None:
        return out
    price = pd.to_numeric(out[close_col], errors='coerce')
    out['sma_10'] = price.rolling(window=10, min_periods=1).mean()
    out['sma_20'] = price.rolling(window=20, min_periods=1).mean()
    out['sma_50'] = price.rolling(window=50, min_periods=1).mean()
    out['ema_12'] = price.ewm(span=12, adjust=False, min_periods=1).mean()
    out['ema_26'] = price.ewm(span=26, adjust=False, min_periods=1).mean()
    # RSI(14)
    delta = price.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=14, min_periods=14).mean()
    avg_loss = loss.rolling(window=14, min_periods=14).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    out['rsi_14'] = 100 - (100 / (1 + rs))
    return out

def load_features(symbol: str, base_dir: str = DATA_DIR) -> pd.DataFrame:
    path = os.path.join(base_dir, f'{symbol}_features.csv')
    if not os.path.exists(path):
        return pd.DataFrame()
    try:
        if os.path.getsize(path) == 0:
            print(f'Warning: features CSV is empty: {path}')
            return pd.DataFrame()
    except OSError:
        pass
    try:
        df = pd.read_csv(path)
    except EmptyDataError:
        print(f'Warning: features CSV contains no parseable data: {path}')
        return pd.DataFrame()
    # If features has a date-like column, align on it
    for c in df.columns:
        if 'date' in str(c).lower():
            df[c] = pd.to_datetime(df[c], errors='coerce', utc=True)
            df = df.dropna(subset=[c]).sort_values(by=c).reset_index(drop=True).set_index(c)
            return df
    df.index.name = 'index'
    return df

def load_dividends(symbol: str, base_dir: str = DATA_DIR) -> pd.DataFrame:
    path = os.path.join(base_dir, f'{symbol}_dividends.csv')
    if not os.path.exists(path):
        return pd.DataFrame()
    try:
        if os.path.getsize(path) == 0:
            print(f'Warning: dividends CSV is empty: {path}')
            return pd.DataFrame()
    except OSError:
        pass
    try:
        df = pd.read_csv(path)
    except EmptyDataError:
        print(f'Warning: dividends CSV contains no parseable data: {path}')
        return pd.DataFrame()
    # Expect columns like ['date', 'dividend']
    date_col = None
    for c in df.columns:
        if 'date' in str(c).lower():
            date_col = c
            break
    if date_col:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce', utc=True)
        df = df.dropna(subset=[date_col]).sort_values(by=date_col).reset_index(drop=True).set_index(date_col)
    else:
        df.index.name = 'index'
    return df

In [5]:
# Load data
hist = load_history(SYMBOL)
feat = load_features(SYMBOL)
if feat.empty:
    feat = compute_indicators(hist)
divs = load_dividends(SYMBOL)

# Merge features into history on index
df = hist.join(feat[[c for c in feat.columns if c not in hist.columns]], how='left')
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,sma_10,sma_20,sma_50,ema_12,ema_26,rsi_14
index,,,,,,,,,,,,,
0,214.460007,221.479996,214.210007,220.320007,81525200,0.0,0.0,220.320007,220.320007,220.320007,220.320007,220.320007,NaN
1,218.750000,219.089996,211.009995,213.210007,59301200,0.0,0.0,216.765007,216.765007,216.765007,219.226161,219.793341,NaN
2,213.250000,215.660004,206.940002,209.210007,62821400,0.0,0.0,214.246674,214.246674,214.246674,217.685214,219.009390,NaN
3,209.720001,211.839996,202.589996,205.750000,64116400,0.0,0.0,212.122505,212.122505,212.122505,215.849027,218.027213,NaN
4,209.800003,214.889999,205.970001,206.279999,62308800,0.0,0.0,210.954004,210.954004,210.954004,214.376869,217.157049,NaN


In [6]:
# Plot price with SMA/EMA overlays and RSI
close_col = None
for c in df.columns:
    if str(c).lower() in ('close', 'adj close', 'adj_close', 'adjclose'):
        close_col = c
        break
if close_col is None:
    raise ValueError('No close-like column found in history CSV')

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.08, row_heights=[0.72, 0.28])

fig.add_trace(go.Scatter(x=df.index, y=df[close_col], name='Close', line=dict(color='#1f77b4')), row=1, col=1)
for name, color in [('sma_10', '#ff7f0e'), ('sma_20', '#2ca02c'), ('sma_50', '#d62728'), ('ema_12', '#9467bd'), ('ema_26', '#8c564b')]:
    if name in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df[name], name=name.upper(), line=dict(width=1)), row=1, col=1)

# RSI panel
if 'rsi_14' in df.columns:
    fig.add_trace(go.Scatter(x=df.index, y=df['rsi_14'], name='RSI(14)', line=dict(color='#17becf')), row=2, col=1)
    # Overbought/oversold lines
    fig.add_hline(y=70, line_dash='dot', line_color='red', row=2, col=1)
    fig.add_hline(y=30, line_dash='dot', line_color='green', row=2, col=1)

# Optional: dividend markers if dates align (both indexed by datetime)
if not divs.empty and isinstance(df.index, pd.DatetimeIndex) and isinstance(divs.index, pd.DatetimeIndex):
    # Drop to common range
    common = divs.index.intersection(df.index)
    if 'dividend' in divs.columns and len(common) > 0:
        y_vals = df.loc[common, close_col]
        fig.add_trace(
            go.Scatter(
                x=common,
                y=y_vals,
                mode='markers',
                name='Dividends',
                marker=dict(color='gold', size=8, symbol='triangle-up')
            ),
            row=1, col=1
        )

fig.update_layout(title=f'{SYMBOL} Price with SMA/EMA and RSI', height=700, legend_orientation='h')
try:
    fig.show()
except Exception as e:
    import plotly.io as pio
    if isinstance(e, ValueError) and 'nbformat' in str(e).lower():
        pio.renderers.default = 'browser'
        print("Renderer fallback: nbformat not available; switched to 'browser'.")
        fig.show()
    else:
        raise

In [7]:
# Plot price with SMA/EMA overlays and MACD
close_col = None
for c in df.columns:
    if str(c).lower() in ('close', 'adj close', 'adj_close', 'adjclose'):
        close_col = c
        break
if close_col is None:
    raise ValueError('No close-like column found in history CSV')

# Compute MACD inputs (ema12, ema26) if missing
price = pd.to_numeric(df[close_col], errors='coerce')
ema12 = df['ema_12'] if 'ema_12' in df.columns else price.ewm(span=12, adjust=False, min_periods=1).mean()
ema26 = df['ema_26'] if 'ema_26' in df.columns else price.ewm(span=26, adjust=False, min_periods=1).mean()
macd_line = ema12 - ema26
signal_line = macd_line.ewm(span=9, adjust=False, min_periods=1).mean()
macd_hist = macd_line - signal_line

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.08, row_heights=[0.72, 0.28])

# Price panel with overlays
fig.add_trace(go.Scatter(x=df.index, y=df[close_col], name='Close', line=dict(color='#1f77b4')), row=1, col=1)
for name, color in [('sma_10', '#ff7f0e'), ('sma_20', '#2ca02c'), ('sma_50', '#d62728'), ('ema_12', '#9467bd'), ('ema_26', '#8c564b')]:
    if name in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df[name], name=name.upper(), line=dict(width=1)), row=1, col=1)

# MACD panel
fig.add_trace(go.Scatter(x=df.index, y=macd_line, name='MACD', line=dict(color='#17becf')), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=signal_line, name='Signal', line=dict(color='#d62728')), row=2, col=1)
# Histogram colored by sign
hist_colors = ['rgba(38,166,91,0.6)' if v >= 0 else 'rgba(219,68,55,0.6)' for v in macd_hist.fillna(0)]
fig.add_trace(go.Bar(x=df.index, y=macd_hist, name='MACD Hist', marker_color=hist_colors), row=2, col=1)
fig.add_hline(y=0, line_dash='dot', line_color='gray', row=2, col=1)

fig.update_layout(title=f'{SYMBOL} Price with SMA/EMA and MACD', height=700, legend_orientation='h')
try:
    fig.show()
except Exception as e:
    import plotly.io as pio
    if isinstance(e, ValueError) and 'nbformat' in str(e).lower():
        pio.renderers.default = 'browser'
        print("Renderer fallback: nbformat not available; switched to 'browser'.")
        fig.show()
    else:
        raise